## European Cities 2D [Part 1]

In [25]:
import numpy as np
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import torch.optim as optim

paris_coords = np.array([
    [48.8575, 2.3514], #Center of Paris
    [48.8584, 2.2945], # Eiffel Tower
    [48.8530, 2.3499], #Notre Dame
    [48.8606,  2.3376], #Louvre
    [48.8606, 2.3522]  #Centre Pompidou
])

madrid_coords = np.array([
    [40.4167, -3.7033],   # Center of Madrid
    [40.4153, -3.6835],   # Retiro Park 
    [40.4180, -3.7143],   # Royal Palace 
    [40.4138, -3.6921],   # Prado Museum 
    [40.4169, -3.7033]   # Puerta del Sol 
])

berlin_coords = np.array([
    [52.5200, 13.4050], # Center of Berlin
    [52.5163, 13.3777],   # Brandenburg Gate 
    [52.5169, 13.4019],   # Museum Island 
    [52.5074, 13.3904],   # Checkpoint Charlie 
    [52.5251, 13.3694]   # Berlin Central Station 
])

barcelona_coords = np.array([
    [41.3874, 2.1686],    # Center of Barcelona
    [41.4036, 2.1744],    # Sagrada Familia
    [41.3819, 2.1773],    # Gothic Quarter 
    [41.4145, 2.1527],    # Park Güell 
    [41.3809, 2.1228],    # Camp Nou
])


In [33]:
mean_to_subtract=np.array([48.8575, 2.3514]) #Center of Paris

In [34]:
# [28, 2.2]

In [35]:
# Set random seed for reproducibility
random_seed=25
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [36]:
# Combine data into one matrix X and labels y 
X_raw = np.vstack([madrid_coords, paris_coords, berlin_coords, barcelona_coords]) #, brussels_coords, vienna_coords])
y = np.array([0, 0, 0, 0, 0,  #  Madrid labels (0)
              1, 1, 1, 1, 1,  #  Paris labels (1)
              2, 2, 2, 2, 2, # Berlin labels (2)
              3, 3, 3, 3, 3]) # Barcelona Labels 
              #3, 3, 3, 3, 3, 3, 3, 
              #4, 4, 4, 4, 4, 4, 4]) 

# Let's use only longitude as input for the simple model
# X_lon = X_raw[:, 1].reshape(-1, 1)  # Extract longitude and reshape to column vector

# Normalize data (simple scaling by dividing by 100, I guess I could do 10)
X = X_raw # / 100 - Simple longitude only problem does not seem to require normalization, that's interesting!
X = X_raw - mean_to_subtract #np.array([28, 2.2]) #Ok looks like I might need to normalize liek this for this one to converge

# Alternative normalization (uncomment to use)
# mean = np.mean(X_lon)
# std = np.std(X_lon)
# X = (X_lon - mean) / std

rI=np.arange(len(y))
np.random.shuffle(rI)
X=X[rI,:]
y=y[rI]
X_raw=X_raw[rI,:]

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Shape of X: (20, 2)
Shape of y: (20,)


In [37]:
X.max(0)

array([ 3.6676, 11.0536])

In [38]:
X.min(0)

array([-8.4437, -6.0657])

xs.shape

In [6]:
class TinyGPSModel(nn.Module):
    def __init__(self, input_size=2, output_size=4):
        super(TinyGPSModel, self).__init__()
        self.output = nn.Linear(input_size, output_size) #, bias=False)  # 3 cities

    def forward(self, x):
        x = self.output(x)
        return x

In [7]:
# Initialize model, loss, and optimizer
model = TinyGPSModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

#I can manually initialize and still learns real good? yeah seems like it!
# with torch.no_grad():
#     model.output.weight[0,0]=1.0
#     model.output.weight[1,0]=0.0
#     model.output.weight[2,0]=-1.0
#     model.output.bias[0]=0
#     model.output.bias[1]=0
#     model.output.bias[2]=0

In [8]:
#Where to save training heatmaps
save_dir='/Users/stephen/Stephencwelch Dropbox/Stephen Welch/welch_labs/backprop2/graphics/to_manim/may_28_2'

In [9]:
#Swaggin - need to sync with Sam on exact numbers
min_long=-6.0
max_long=17.0
min_lat=36.0
max_lat=56.0
num_steps=256
heatmap_viz_logit_multiplier=8 #Makes things more winner take all for cleaner logit viz
heatmaps=[np.zeros((num_steps, num_steps)) for i in range(8)]

for i, lat in enumerate(np.linspace(max_lat, min_lat, num_steps)):
    for j, long in enumerate(np.linspace(min_long, max_long, num_steps)):
        with torch.no_grad():
            logits=model(torch.tensor([lat, long], dtype=torch.float)).detach()
            yhat=torch.nn.Softmax(0)(heatmap_viz_logit_multiplier*logits).numpy()

        for k in range(4):
            heatmaps[k][i,j]=logits.numpy()[k]
            heatmaps[k+3][i,j]=yhat[k]

In [41]:
min_lat-mean_to_subtract[0], max_lat-mean_to_subtract[0]

(-12.857500000000002, 7.142499999999998)

In [44]:
min_long-mean_to_subtract[1], max_long-mean_to_subtract[1]

(-8.3514, 14.6486)

In [ ]:
#-13, 6, -8, 15

In [10]:
import matplotlib.colors as mcolors
def create_transparent_colormap(color='cyan', name='transparent_to_color'):
    # Convert color name to RGBA
    base_color = mcolors.to_rgba(color)
    
    # Create colormap: transparent (alpha=0) to full color (alpha=1)
    colors = [(base_color[0], base_color[1], base_color[2], 0),  # transparent
              (base_color[0], base_color[1], base_color[2], 1)]  # full color
    
    n_bins = 256
    cmap = mcolors.LinearSegmentedColormap.from_list(name, colors, N=n_bins)
    return cmap

# Create the colormap
transparent_cyan_cmap = create_transparent_colormap('cyan')
transparent_yellow_cmap = create_transparent_colormap('#ffd35a')
transparent_green_cmap = create_transparent_colormap('#00a14b')
transparent_magenta_cmap = create_transparent_colormap('magenta')

In [11]:
cmaps=[transparent_cyan_cmap, transparent_yellow_cmap, transparent_green_cmap, transparent_magenta_cmap, 
       transparent_cyan_cmap, transparent_yellow_cmap, transparent_green_cmap, transparent_magenta_cmap]
save_names=['_logits_1.png', '_logits_2.png', '_logits_3.png', '_logits_4.png', 
            '_yhat_1.png', '_yhat_2.png', '_yhat_3.png', '_yhat_4.png']

# for l in range(6):
#     plt.clf()
#     plt.figure(frameon=False)
#     ax = plt.Axes(plt.gcf(), [0., 0., 1., 1.])
#     ax.set_axis_off()
#     plt.gcf().add_axes(ax)
#     plt.imshow(heatmaps[l],  cmap=cmaps[l]) #np.rot90(heatmaps[0])) #Wait and see if I need to rotate or transpose
#     plt.savefig(save_dir+save_names[l], bbox_inches='tight', pad_inches=0, dpi=300)
#     plt.close()

In [12]:
# Initialize model, loss, and optimizer
model = TinyGPSModel()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.5)
optimizer = optim.Adam(model.parameters(), lr=0.1)

#I can manually initialize and still learns real good? yeah seems like it!
# with torch.no_grad():
#     model.output.weight[0,0]=1.0
#     model.output.weight[1,0]=0.0
#     model.output.weight[2,0]=-1.0
#     model.output.bias[0]=0
#     model.output.bias[1]=0
#     model.output.bias[2]=0

weights=[]
grads=[]
xs=[]
ys=[]
logitss=[]
yhats=[]

# Training loop
for i in range(350):
    xs.append(X_raw[i%len(y)])
    ys.append(y[i%len(y)])
    weights.append(np.concatenate([model.output.weight.detach().numpy().ravel(), model.output.bias.detach().numpy().ravel()]))
    
    optimizer.zero_grad()
    
    #Stochastic - i think this is a better starting point pedagogically. 
    outputs = model(torch.tensor(X[i%len(y)]).float())
    loss = criterion(outputs, torch.tensor(y[i%len(y)])) 

    logitss.append(outputs.detach().numpy())
    yhats.append(torch.nn.Softmax(0)(outputs.detach()).numpy())
    
    loss.backward()  # backpropagation
    grads.append(np.concatenate([model.output.weight.grad.detach().numpy().ravel(), model.output.bias.grad.detach().numpy().ravel()]))
    optimizer.step() #
    
    if (i + 1) % 10 == 0:
        with torch.no_grad():
            logits=model(torch.tensor(X, dtype=torch.float)) 
            accuracy=(torch.argmax(logits, dim=1)==torch.tensor(y)).sum().item()/len(y)
        print(f"Step {i+1}/{i}, Loss: {loss.item():.4f}, 'Accuracy: {accuracy:.4f}")

weights=np.array(weights)
grads=np.array(grads)
xs=np.array(xs)
ys=np.array(ys)
logitss=np.array(logitss)
yhats=np.array(yhats)

Step 10/9, Loss: 2.3401, 'Accuracy: 0.5000
Step 20/19, Loss: 2.5776, 'Accuracy: 0.5000
Step 30/29, Loss: 1.6629, 'Accuracy: 0.7500
Step 40/39, Loss: 0.0103, 'Accuracy: 0.7500
Step 50/49, Loss: 0.8456, 'Accuracy: 1.0000
Step 60/59, Loss: 0.0580, 'Accuracy: 1.0000
Step 70/69, Loss: 0.4112, 'Accuracy: 1.0000
Step 80/79, Loss: 0.0151, 'Accuracy: 1.0000
Step 90/89, Loss: 0.2313, 'Accuracy: 1.0000
Step 100/99, Loss: 0.0102, 'Accuracy: 1.0000
Step 110/109, Loss: 0.1541, 'Accuracy: 1.0000
Step 120/119, Loss: 0.0105, 'Accuracy: 1.0000
Step 130/129, Loss: 0.1137, 'Accuracy: 1.0000
Step 140/139, Loss: 0.0098, 'Accuracy: 1.0000
Step 150/149, Loss: 0.0891, 'Accuracy: 1.0000
Step 160/159, Loss: 0.0085, 'Accuracy: 1.0000
Step 170/169, Loss: 0.0726, 'Accuracy: 1.0000
Step 180/179, Loss: 0.0074, 'Accuracy: 1.0000
Step 190/189, Loss: 0.0608, 'Accuracy: 1.0000
Step 200/199, Loss: 0.0065, 'Accuracy: 1.0000
Step 210/209, Loss: 0.0519, 'Accuracy: 1.0000
Step 220/219, Loss: 0.0058, 'Accuracy: 1.0000
Step 230

In [13]:
# Initialize model, loss, and optimizer
model = TinyGPSModel()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.03)

#I can manually initialize and still learns real good? yeah seems like it!
# with torch.no_grad():
#     model.output.weight[0,0]=1.0
#     model.output.weight[1,0]=0.0
#     model.output.weight[2,0]=-1.0
#     model.output.bias[0]=0
#     model.output.bias[1]=0
#     model.output.bias[2]=0

weights=[]
grads=[]
xs=[]
ys=[]
logitss=[]
yhats=[]

# Training loop
for i in range(500):
    xs.append(X_raw[i%len(y)])
    ys.append(y[i%len(y)])
    weights.append(np.concatenate([model.output.weight.detach().numpy().ravel(), model.output.bias.detach().numpy().ravel()]))
    
    optimizer.zero_grad()
    
    #Stochastic - i think this is a better starting point pedagogically. 
    outputs = model(torch.tensor(X[i%len(y)]).float())
    loss = criterion(outputs, torch.tensor(y[i%len(y)])) 

    logitss.append(outputs.detach().numpy())
    yhats.append(torch.nn.Softmax(0)(outputs.detach()).numpy())

    #Heatmaps
    heatmaps=[np.zeros((num_steps, num_steps)) for i in range(8)]
    for j, lat in enumerate(np.linspace(max_lat, min_lat, num_steps)):
        for k, long in enumerate(np.linspace(min_long, max_long, num_steps)):
            with torch.no_grad():
                coords_norm=np.array([lat, long])-mean_to_subtract
                logits=model(torch.tensor(coords_norm.ravel(), dtype=torch.float)).detach()
                yhat=torch.nn.Softmax(0)(heatmap_viz_logit_multiplier*logits).numpy()
    
            for l in range(4):
                heatmaps[l][j, k]=logits.numpy()[l]
                heatmaps[l+4][j,k]=yhat[l]

    for l in range(8):
        plt.clf()
        plt.figure(frameon=False)
        ax = plt.Axes(plt.gcf(), [0., 0., 1., 1.])
        ax.set_axis_off()
        plt.gcf().add_axes(ax)
        plt.imshow(heatmaps[l],  cmap=cmaps[l]) #np.rot90(heatmaps[0])) #Wait and see if I need to rotate or transpose
        plt.savefig(save_dir+'/'+str(i)+save_names[l], bbox_inches='tight', pad_inches=0, dpi=300)
        plt.close()
    
    loss.backward()  # backpropagation
    grads.append(np.concatenate([model.output.weight.grad.detach().numpy().ravel(), model.output.bias.grad.detach().numpy().ravel()]))
    optimizer.step() #
    
    if (i + 1) % 10 == 0:
        with torch.no_grad():
            logits=model(torch.tensor(X, dtype=torch.float)) 
            accuracy=(torch.argmax(logits, dim=1)==torch.tensor(y)).sum().item()/len(y)
        print(f"Step {i+1}/{i}, Loss: {loss.item():.4f}, 'Accuracy: {accuracy:.4f}")

weights=np.array(weights)
grads=np.array(grads)
xs=np.array(xs)
ys=np.array(ys)
logitss=np.array(logitss)
yhats=np.array(yhats)

Step 10/9, Loss: 1.3858, 'Accuracy: 0.7500
Step 20/19, Loss: 0.1424, 'Accuracy: 0.7500
Step 30/29, Loss: 1.0455, 'Accuracy: 0.7500
Step 40/39, Loss: 0.0178, 'Accuracy: 1.0000
Step 50/49, Loss: 0.7605, 'Accuracy: 1.0000
Step 60/59, Loss: 0.0174, 'Accuracy: 1.0000
Step 70/69, Loss: 0.5511, 'Accuracy: 1.0000
Step 80/79, Loss: 0.0148, 'Accuracy: 1.0000
Step 90/89, Loss: 0.4075, 'Accuracy: 1.0000
Step 100/99, Loss: 0.0105, 'Accuracy: 1.0000
Step 110/109, Loss: 0.3102, 'Accuracy: 1.0000
Step 120/119, Loss: 0.0084, 'Accuracy: 1.0000
Step 130/129, Loss: 0.2433, 'Accuracy: 1.0000
Step 140/139, Loss: 0.0071, 'Accuracy: 1.0000
Step 150/149, Loss: 0.1960, 'Accuracy: 1.0000
Step 160/159, Loss: 0.0061, 'Accuracy: 1.0000
Step 170/169, Loss: 0.1616, 'Accuracy: 1.0000
Step 180/179, Loss: 0.0052, 'Accuracy: 1.0000
Step 190/189, Loss: 0.1359, 'Accuracy: 1.0000
Step 200/199, Loss: 0.0046, 'Accuracy: 1.0000
Step 210/209, Loss: 0.1162, 'Accuracy: 1.0000
Step 220/219, Loss: 0.0041, 'Accuracy: 1.0000
Step 230

<Figure size 640x480 with 0 Axes>

In [14]:
logits.numpy()

array([[-24.69661   ,  -6.039481  ,   6.041276  ,  -1.5216051 ],
       [-24.670286  ,  -6.011381  ,   6.038615  ,  -1.532564  ],
       [  9.720504  ,  -0.9818263 ,  -8.172711  ,   3.0570273 ],
       [-24.725891  ,  -6.0406933 ,   6.054416  ,  -1.5273451 ],
       [ -2.1891336 ,   2.9525757 ,  -1.3279425 ,  -1.9197239 ],
       [ -2.2206638 ,  -7.006975  ,  -4.6881056 ,   4.0168843 ],
       [ -2.2654593 ,  -7.0491247 ,  -4.6816573 ,   4.0321503 ],
       [  9.697978  ,  -0.9974375 ,  -8.167577  ,   3.0613718 ],
       [  9.674975  ,  -1.01548   ,  -8.163045  ,   3.0670614 ],
       [ -2.3025289 ,   2.8755128 ,  -1.3015772 ,  -1.8987647 ],
       [  9.6574745 ,  -1.0242581 ,  -8.15792   ,   3.0684366 ],
       [ -2.3053987 ,   2.879754  ,  -1.298812  ,  -1.9019293 ],
       [-24.652952  ,  -5.9889517 ,   6.0381927 ,  -1.542123  ],
       [ -2.306899  ,   2.8829694 ,  -1.2970282 ,  -1.9041792 ],
       [ -2.1610386 ,  -7.015114  ,  -4.718457  ,   4.034902  ],
       [ -2.2723193 ,  -7

In [15]:
all_training_data=np.hstack((xs, ys.reshape(-1, 1), weights, grads, logitss, yhats))
np.save('/Users/stephen/Stephencwelch Dropbox/Stephen Welch/welch_labs/backprop2/hackin/cities_2d_2', all_training_data)

In [16]:
all_training_data.shape

(500, 35)

In [17]:
xs.shape

(500, 2)

In [18]:
ys.shape

(500,)

In [19]:
weights.shape

(500, 12)

In [20]:
grads.shape

(500, 12)

In [21]:
logitss.shape

(500, 4)

In [22]:
yhats.shape

(500, 4)

In [23]:
model.output.weight.detach().numpy().ravel()

array([ 0.04314625, -2.042639  ,  1.361917  , -1.2582749 ,  0.4415128 ,
        0.5189429 , -0.8032285 ,  0.3000297 ], dtype=float32)

In [24]:
model.output.weight.detach().numpy()

array([[ 0.04314625, -2.042639  ],
       [ 1.361917  , -1.2582749 ],
       [ 0.4415128 ,  0.5189429 ],
       [-0.8032285 ,  0.3000297 ]], dtype=float32)